In [23]:
from bs4 import BeautifulSoup
import sys
import os
import requests
import re
from anyascii import anyascii
import pandas as pd
year = sys.argv[1] if len(sys.argv) > 1 else '1760'
print(year)


# https://caselaw.findlaw.com/court/us-supreme-court/years/1760



--ip=127.0.0.1


In [24]:
year = '2008'

In [25]:
soup = BeautifulSoup(requests.get('https://caselaw.findlaw.com/court/us-supreme-court/years/'+year).text, 'html.parser')

In [26]:
description = soup.find_all(attrs={"data-label": "Description"})

In [27]:
links = []
for item in description:
    links.append(item.a.get('href'))

In [28]:
links

['https://caselaw.findlaw.com/us-supreme-court/552/248.html',
 'https://caselaw.findlaw.com/us-supreme-court/552/214.html',
 'https://caselaw.findlaw.com/us-supreme-court/553/662.html',
 'https://caselaw.findlaw.com/us-supreme-court/555/70.html',
 'https://caselaw.findlaw.com/us-supreme-court/555/57.html',
 'https://caselaw.findlaw.com/us-supreme-court/07-818.html',
 'https://caselaw.findlaw.com/us-supreme-court/08-5385.html',
 'https://caselaw.findlaw.com/us-supreme-court/552/117.html',
 'https://caselaw.findlaw.com/us-supreme-court/553/35.html',
 'https://caselaw.findlaw.com/us-supreme-court/6-11543.html',
 'https://caselaw.findlaw.com/us-supreme-court/552/421.html',
 'https://caselaw.findlaw.com/us-supreme-court/553/723.html',
 'https://caselaw.findlaw.com/us-supreme-court/553/639.html',
 'https://caselaw.findlaw.com/us-supreme-court/553/124.html',
 'https://caselaw.findlaw.com/us-supreme-court/553/442.html',
 'https://caselaw.findlaw.com/us-supreme-court/554/60.html',
 'https://cas

In [29]:
outList = []

In [30]:
for link in links:
    print('Fetching '+link)
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    dates = soup.find_all('h3')[2].get_text().replace('Argued:','').partition('Decided:')
    line = [soup.find_all('h3')[0].get_text(),soup.find_all('h3')[1].get_text(),dates[0].strip(),dates[2].strip()]
    line.append(anyascii('\n'.join([someTxt.get_text().strip().replace('\xa0',' ').replace('\n',' ') for someTxt in soup.find_all(class_='caselawcontent searchable-content')])))
    outList.append(line)

Fetching https://caselaw.findlaw.com/us-supreme-court/552/248.html
Fetching https://caselaw.findlaw.com/us-supreme-court/552/214.html
Fetching https://caselaw.findlaw.com/us-supreme-court/553/662.html
Fetching https://caselaw.findlaw.com/us-supreme-court/555/70.html
Fetching https://caselaw.findlaw.com/us-supreme-court/555/57.html
Fetching https://caselaw.findlaw.com/us-supreme-court/07-818.html
Fetching https://caselaw.findlaw.com/us-supreme-court/08-5385.html
Fetching https://caselaw.findlaw.com/us-supreme-court/552/117.html
Fetching https://caselaw.findlaw.com/us-supreme-court/553/35.html
Fetching https://caselaw.findlaw.com/us-supreme-court/6-11543.html
Fetching https://caselaw.findlaw.com/us-supreme-court/552/421.html
Fetching https://caselaw.findlaw.com/us-supreme-court/553/723.html
Fetching https://caselaw.findlaw.com/us-supreme-court/553/639.html
Fetching https://caselaw.findlaw.com/us-supreme-court/553/124.html
Fetching https://caselaw.findlaw.com/us-supreme-court/553/442.html

In [31]:

df = pd.DataFrame(outList,columns=['Title','Docket','Argued','Decided','Text'])

In [32]:
df

,Title,Docket,Argued,Decided,Text
0,"LARUE v. DEWOLFF, BOBERG & ASSOCIATES, INC., E...",No. 06-856,"November 26, 2007","February 20, 2008","United States Supreme Court LARUE v. DEWOLFF, ..."
1,ALI v. FEDERAL BUREAU OF PRISONS ET AL.(2008),No. 06-9130,"October 29, 2007","January 22, 2008",United States Supreme Court ALI v. FEDERAL BUR...
2,"ALLISON ENGINE CO., INC., ET AL. v. UNITED STA...",No. 07-214,"February 26, 2008","June 9, 2008",United States Supreme Court ALLISON ENGINE CO....
3,"ALTRIA GROUP, INC., ET AL. v. GOOD ET AL.(2008)",No. 07-562,"October 6, 2008","December 15, 2008","United States Supreme Court ALTRIA GROUP, INC...."
4,"ANTHONY HEDGPETH, WARDEN, PETITIONER v. MICH...",No. 07-544,,"December 2, 2008","United States Supreme Court ANTHONY HEDGPETH, ..."
...,...,...,...,...,...
77,UNITED STATES v. WILLIAMS(2008),No. 06-694,"October 30, 2007","May 19, 2008",United States Supreme Court UNITED STATES v. W...
78,VIRGINIA v. MOORE(2008),No. 06-1082,"January 14, 2008","April 23, 2008",United States Supreme Court VIRGINIA v. MOORE(...
79,"WARNER-LAMBERT CO., LLC, ET AL., PETITIONERS ...",No. 06-1498,,"March 3, 2008",United States Supreme Court WARNER-LAMBERT CO....
80,WASHINGTON STATE GRANGE v. WASHINGTON STATE RE...,No. 06-713,"October 1, 2007","March 18, 2008",United States Supreme Court WASHINGTON STATE G...


In [33]:
# with open('data/'+year+'.csv','w') as f:
#     for line in outList:
#         f.write(','.join(line)+'\n')
if len(outList)>0:
    df.to_pickle('data/'+year+'.pkl.zst',compression='zstd')
    print('File written to data/'+year+'.pkl.zst')
else:
    print('Skipped '+year)

File written to data/2008.pkl.zst
